### Import Library

In [1]:
import pandas as pd
import re
import ast
import os

pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)        # Disable line wrapping
pd.set_option('display.max_colwidth', None) # Show long text fully

### Load Data

In [2]:
# df_res = pd.read_csv('一般訂單.csv')
df_un = pd.read_csv(r'C:\Users\USER\OneDrive\Documents\DSC_Project\20250517_資料集\customization.csv')
df_raw = df_un
folder_name = 'costumization_clean'

### Define Fnution

In [3]:
# The dataset except for numeric are all strings, so we have to convert into python objects.

# Checking for missing commas in the list of dictionaries
def fix_missing_commas(raw_string):
    """
    Insert commas between dictionaries in lists, e.g., [{...}{...}] -> [{...}, {...}]
    """
    fixed_string = re.sub(r'\}[\s]*\{', '}, {', raw_string)
    return fixed_string

def parse_json_like(val):
    """
    將 string 讀成 Python 物件
    """
    try:
        if isinstance(val, str):
            val = fix_missing_commas(val)
            fixed = re.sub(r'\}[\s]*\{', '}, {', val)
            #處理時間資料
            fixed_time = re.sub(r'Timestamp\((.*?)\)', r'\1', fixed)
            #處理array型式
            cleaned = re.sub(r'array\((\[.*?\])\s*,\s*dtype=.*?\)', r'\1', fixed_time)

            return ast.literal_eval(cleaned)
        else:
            return val
    except Exception as e:
        #print(str_val)
        #print(f"Error parsing services : {e}")
        return val
    
def extend_dict_like(df, col_name, converted=False, rename=True):
    """
    Extend the dictionary-like column into separate columns.
    """
    if not converted:
        df_parsed = df[col_name].apply(parse_json_like)
    else:
        df_parsed = df[col_name]
    df_expanded = df_parsed.apply(lambda x: pd.Series(x))

    try:
        df_expanded.drop([0], axis=1, inplace=True) # Drops any column named 0 (likely a bug fix for unexpected data).
    except KeyError:
        pass

    if rename:
        df_expanded.columns = [col_name + "_" + str(key) for key in df_expanded.columns]
    return df_expanded

def extract_dict_list_like(df, col_name, id_col):
    """
    Convert a column of lists of dictionaries into a separate table.
    """
    df_current = df[[id_col, col_name]].rename(columns = {id_col: 'order_id'})
    df_exploded = df_current.explode(col_name)
    detailed_df = extend_dict_like(df_exploded, col_name, converted = True, rename = False)

    try:
        detailed_df.drop([0], axis=1, inplace=True) # Drops any column named 0 (likely a bug fix for unexpected data).
    except KeyError:
        pass

    df_final = pd.concat([df_exploded['order_id'], detailed_df] ,axis=1)

    return df_final


### Main Table Cleaning

In [4]:
df_parsed = df_raw.applymap(parse_json_like)  # Parse all cells
df = df_parsed.copy()

error_columns = []
subDF_dict = {}

loops = 0
while True:
    loops += 1
    flag = 0
    print(f"Loop {loops}")

    for col in df.columns:
        if col in error_columns:
            continue

        if df[col].isna().all():
            continue
        else:
            sample = df[col].dropna().iloc[0]

        if isinstance(sample, dict):
            print(f"Expanding dict : {col}")
            flag = 1
            try:
                col_expend = extend_dict_like(df, col, converted = True)
                df = pd.concat([df, col_expend], axis=1)
                df.drop([col], axis=1, inplace=True)
            except:
                print(f"Problem processing {col} ... added to error_columns list")
                error_columns.append(col)
        elif isinstance(sample, list):
            print(f"Isolating a new subtable for column: {col}")
            try:
                expanded_df = extract_dict_list_like(df, col, "_id_oid")
                subDF_dict[col] = expanded_df
                df.drop(col, axis=1, inplace=True)
            except:
                print(f"Problem processing {col} ... added to error_columns list")
                error_columns.append(col)
    
    if flag == 0:
        print("No more columns to process.")
        break

# Save main table
os.makedirs(folder_name, exist_ok=True)
df.to_csv(f"{folder_name}/main.csv", index=False)

# Save sub-tables
for sub_table, sub_df in subDF_dict.items():
    sub_df.to_csv(f"{folder_name}/{sub_table}.csv", index=False)

Loop 1
Isolating a new subtable for column: items
Problem processing items ... added to error_columns list
No more columns to process.


C:\Users\USER\AppData\Local\Temp\ipykernel_15252\3560262479.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_parsed = df_raw.applymap(parse_json_like)  # Parse all cells


### SubTable Cleaning

In [5]:
error_columns = []
for subt_table in subDF_dict.keys():
    print("\n=====\n")
    print(f"Cleaning sub_table {sub_table}")
    df_cleaned = subDF_dict[sub_table]
    df_cleaned.dropna(how='all', inplace=True) # Drop rows where all elements are NaN
    cols = df_cleaned.columns

    loops = 0
    while True:
            loops += 1
            flag = 0
            print(f"Loop : {loops}")
            for col in df_cleaned.columns:
                if df_cleaned[col].isna().all():
                    continue
                else:
                    sample = df_cleaned[col].dropna().iloc[0]
                if type(sample) == type(dict()):
                    print(f"Processing dict : {col}")
                    flag = 1
                    try:
                        col_expanded = extend_dict_like(df_cleaned, col, converted=True)
                        df_cleaned = pd.concat([df_cleaned, col_expanded], axis=1)
                        df_cleaned.drop(col, axis=1, inplace=True)
                    except:
                        print(f"Problem processing {col} ... added to error_columns list")
                        error_columns.append(col)
            if flag == 0:
                break
            
    df_cleaned.to_csv(f"{folder_name}/{sub_table}.csv", index=False)
    print(f"Subtable saved to folder.")

### Documenting the Output

In [6]:
print(f"資料夾名稱: {folder_name}")
print("\n======\n")
print("主表: main.csv\n")
print(f"副表列表 ({len(subDF_dict.keys())}):")
for key in subDF_dict.keys():
    print(f"-- {key}.csv")
print("\n=====\n")
print(f"主表副表連結: 主表的 _id_oid 欄位對上 副表的 order_id_oid 欄位")

with open(f"{folder_name}\Description.txt", "w", encoding="utf-8") as f:
    f.write(f"資料夾名稱: {folder_name}\n")
    f.write("\n======\n\n")
    f.write("主表: main.csv\n\n")
    f.write(f"副表列表 ({len(subDF_dict.keys())}):\n")
    for key in subDF_dict.keys():
        f.write(f"-- {key}.csv\n")
    f.write("\n=====\n\n")
    f.write("主表副表連結: 主表的 _id_oid 欄位對上 副表的 order_id_oid 欄位\n")

資料夾名稱: costumization_clean


主表: main.csv

副表列表 (0):

=====

主表副表連結: 主表的 _id_oid 欄位對上 副表的 order_id_oid 欄位
